In [1]:
#This section will deal with importing the data from the .json files and processing it into pandas dataframes

In [2]:
import json
import pandas as pd
import matplotlib.pyplot as plt

# open input file: 
reviewFile = open('review.json', encoding="utf8")
businessFile = open('business.json', encoding="utf8")
userFile = open('user.json', encoding="utf8")

# read the first 500k entries
# set to -1 to process everything
stop = 100000

allDataReview= list()
for i, line in enumerate(reviewFile):
    if i==stop:
        break    
    # converting the json into a dictionary
    data = json.loads(line)
    # extraction of the data we want
    business_id = data['business_id']
    user_id = data['user_id']
    text = data['text']
    stars = data['stars']
    # adding data that we've collected
    allDataReview.append([business_id, user_id, stars, text])

allBusinessReview = list()
for i, line in enumerate(businessFile):
    if i==stop:
        break
    
    data = json.loads(line)
    
    business_id = data['business_id']
    name = data['name']
    city = data['city']
    state = data['state']
    stars = data['stars']
    categories = data['categories']   

    allBusinessReview.append([business_id, name, city, state, stars, categories])
    
allUserReview = list()
for i, line in enumerate(userFile):
    if i==stop:
        break
    
    data = json.loads(line)
    
    user_id = data['user_id']
    review_count = data['review_count']
    
    allUserReview.append([user_id, review_count])
    
# create the DataFrame
dfReview = pd.DataFrame(allDataReview, columns=['business_id', 'user_id', 'stars','text'])
dfBusiness = pd.DataFrame(allBusinessReview, columns=['business_','name', 'city', 'state', 'stars', 'categories'])
dfUser = pd.DataFrame(allUserReview, columns=['user_id', 'review_count'])
print(dfReview)
print(dfBusiness)
print(dfUser)


reviewFile.close()
businessFile.close()
userFile.close()

                  business_id                 user_id  stars  \
0      ujmEBvifdJM6h6RLv4wQIg  hG7b0MtEbXx5QzbzE6C_VA    1.0   
1      NZnhc2sEQy3RmzKTZnqtwQ  yXQM5uF2jS6es16SJzNHfg    5.0   
2      WTqjgwHlXbSFevF32_DJVw  n6-Gk65cPZL6Uz8qRm3NYw    5.0   
3      ikCg8xy5JIg_NGPx-MSIDA  dacAIZ6fTM6mqwW5uxkskg    5.0   
4      b1b1eb3uo-w561D0ZfCEiQ  ssoyf2_x0EQMed6fgHeMyQ    1.0   
...                       ...                     ...    ...   
99995  PeU2rTATIarPVjOyzvY-Fg  kZA-neTnGq6gzLt7k70kEQ    3.0   
99996  YRyYbOSwvHkZsZOLv98oQg  7D4A_euL-c2-X9d51TnZtw    1.0   
99997  Der6cNgKbShEMR2jry1Tmg  Mfn6rBrTZg1ysBoknKt1TQ    5.0   
99998  LVG3B79xZfjLcjfJUQId0A  _D9uaoR1I9o2aIEODh4r9A    5.0   
99999  e4h-sBBD70pn1RuAv_Q61w  zemsVfzVyyJDPRg6742H4Q    5.0   

                                                    text  
0      Total bill for this horrible service? Over $8G...  
1      I *adore* Travis at the Hard Rock's new Kelly ...  
2      I have to say that this office really has it t.

In [ ]:
#Removing the stopwords from the text reviews

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')

#first removing punctuation and other characters and making the entire text review lower case
dfReview['text'] = dfReview.text.str.replace("[^\w\s]", "")
dfReview['text'] = dfReview.text.apply(lambda x: x.lower())
dfReview['text'] = dfReview.text.str.replace("‰Ûª", "''")

dfReview['text'] = dfReview['text'].apply(lambda x: ' '.join([item for item in x.split() if item not in stop]))




[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ryanw\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#Successfully removed all the stop words from the text rows

print(dfReview)